# Recommendation System Model Anaylsis
Research on data fetching, modeling, and machine learning for a social media recommendation system

## API Configuration Testing

In [6]:
import sys
sys.path.append('../')

Twitter API Testing

In [7]:
from twitter import TwitterAPI

In [8]:
twitter_api = TwitterAPI()

In [13]:
# Get Retweeters
retweeters = twitter_api.retweeters('1417945380029751296', 5)
retweeters

[1437428060442275858,
 1072697952,
 1436113843206320132,
 937978958855000064,
 988828070747541505]

In [28]:
input_tweets = ['1417945380029751296','1443278205582983168']

In [29]:
tweets = {}
interactors = {}
authors = set()

In [30]:
class Tweet:
    def __init__(self, tweet):
        self.data = tweet
        self.id = tweet['id']
        self.author = tweet['user']['id']
        self.text = tweet['text']
        self.sim_score = 0  # content-based similarity score
        self.collab_score = 0  # user-based collaborative score
        self.overall_score = 0  # DNN final score

In [31]:
# fetch data from each input tweet
for tweet_id in input_tweets:
    tweet_json = twitter_api.tweet_info(tweet_id)
    tweet = Tweet(tweet_json)
    tweets[tweet.id] = tweet

    # add author
    authors.add(tweet.author)

    # add retweeters and update interactions
    retweeters = twitter_api.retweeters(tweet_id, 20)
    for user_id in retweeters:
        if user_id in interactors:
            interactors[user_id].add(tweet.id)
        else:
            interactors[user_id] = {tweet.id}

In [32]:
tweets

{1417945380029751296: <__main__.Tweet at 0x7fdf803c5d30>,
 1443278205582983168: <__main__.Tweet at 0x7fdf80e6f860>}

In [33]:
retweeters[:3]

[1295666711262240768, 1346918022091468800, 1440410676430708744]

In [34]:
# for each author get their /user_timeline and /favorites_list
for author in authors:
    timeline_tweets = twitter_api.user_timeline(author)

    # update interactions w/ author's interactions
    if author in interactors:
        interactors[author].update([tweet['id'] for tweet in timeline_tweets])
    else:
        interactors[author] = set([tweet['id'] for tweet in timeline_tweets])

    # add author's tweets
    for tweet in timeline_tweets:
        if tweet['id'] not in tweets:
            tweets[tweet['id']] = Tweet(tweet)

In [35]:
interactors

{1437428060442275858: {1417945380029751296},
 1072697952: {1417945380029751296},
 1436113843206320132: {1417945380029751296},
 937978958855000064: {1417945380029751296},
 988828070747541505: {1417945380029751296},
 1436884559585447937: {1417945380029751296},
 1237861039829422083: {1417945380029751296},
 1349191347740110849: {1417945380029751296},
 14891619: {1417945380029751296},
 1435477485064318978: {1417945380029751296},
 784543220554489857: {1417945380029751296},
 1398922777093148674: {1417945380029751296},
 20557070: {1417945380029751296},
 1022205261841346560: {1417945380029751296},
 1432356215397433348: {1417945380029751296},
 1347351988653101056: {1417945380029751296},
 1400300466064347137: {1417945380029751296},
 1495493707: {1417945380029751296},
 1428273654664777729: {1417945380029751296},
 1374024046933729280: {1417945380029751296},
 1295666711262240768: {1443278205582983168},
 1346918022091468800: {1443278205582983168},
 1440410676430708744: {1443278205582983168},
 2342689

In [36]:
# for each interactor (retweeters and authors) get their /favorites_list
for user in interactors:
    favorited_tweets = twitter_api.favorite_list(user)
    interactors[user].update([tweet['id'] for tweet in favorited_tweets])

    for tweet in favorited_tweets:
        if tweet['id'] not in tweets:
            tweets[tweet['id']] = Tweet(tweet)